In [1]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Tuple
import logging

In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
@dataclass
class StudyParameters:
    """Class to store and validate study parameters."""
    T: int  # Booking horizon
    N: int  # Service horizon
    C: int  # Room capacity
    price_min: float  # Minimum price
    price_max: float  # Maximum price
    alpha: float  # Smoothing parameter for price acceptance
    beta: float  # Smoothing parameter for capacity
    
    def __post_init__(self):
        """Validate parameters after initialization."""
        if self.T <= 0:
            raise ValueError("Booking horizon T must be positive")
        if self.N <= 0:
            raise ValueError("Service horizon N must be positive")
        if self.C <= 0:
            raise ValueError("Capacity C must be positive")
        if self.price_min >= self.price_max:
            raise ValueError("Minimum price must be less than maximum price")
        if self.alpha <= 0 or self.beta <= 0:
            raise ValueError("Smoothing parameters must be positive")

In [5]:
class DataGenerator:
    """Generate data for hotel dynamic pricing study."""
    
    def __init__(self, params: StudyParameters, seed: int = None):
        """
        Initialize the data generator.
        
        Args:
            params: StudyParameters object containing study parameters
            seed: Random seed for reproducibility
        """
        self.params = params
        self.rng = np.random.default_rng(seed)
        self.booking_classes = self._generate_booking_classes()
        logger.info(f"Initialized DataGenerator with {len(self.booking_classes)} booking classes")
    
    def _generate_booking_classes(self) -> List[Tuple[int, int]]:
        """
        Generate booking classes (arrival day, departure day pairs) with a maximum
        length of stay of 7 nights.
        
        Returns:
            List of tuples (arrival_day, departure_day)
        """
        MAX_LOS = 7  # Maximum length of stay
        booking_classes = []
        for arrival in range(1, self.params.N + 1):
            for los in range(1, min(MAX_LOS + 1, self.params.N - arrival + 2)):
                departure = arrival + los - 1
                booking_classes.append((arrival, departure))
        
        logger.info(f"Generated {len(booking_classes)} booking classes with max LOS = {MAX_LOS}")
        return booking_classes
    
    def generate_arrival_probabilities(self, demand_scenario: str = 'base') -> Dict[int, Dict[Tuple[int, int], float]]:
        """
        Generate arrival probabilities incorporating different demand scenarios.
        
        Args:
            demand_scenario: Type of demand pattern to generate:
                           'base': Normal demand pattern
                           'high': Increased arrival probabilities
                           'low': Decreased arrival probabilities
                           'peak': Time-dependent demand spikes
                           'fluctuating': Higher demand variability
        """
        if demand_scenario not in ['base', 'high', 'low', 'peak', 'fluctuating']:
            raise ValueError(f"Invalid demand scenario: {demand_scenario}")

        # Define scenario-specific parameters
        scenario_params = {
            'base': {
                'demand_multiplier': 1.0,
                'variability': 0.2,
                'peak_periods': []
            },
            'high': {
                'demand_multiplier': 1.5,
                'variability': 0.2,
                'peak_periods': []
            },
            'low': {
                'demand_multiplier': 0.5,
                'variability': 0.2,
                'peak_periods': []
            },
            'peak': {
                'demand_multiplier': 1.0,
                'variability': 0.2,
                'peak_periods': [(self.params.T // 2, 2.0)]  # (period, multiplier)
            },
            'fluctuating': {
                'demand_multiplier': 1.0,
                'variability': 0.4,
                'peak_periods': []
            }
        }

        # Get scenario parameters
        scenario = scenario_params[demand_scenario]
        
        # Define day-of-week seasonality factors
        dow_factors = {
            0: 0.8,  # Sunday
            1: 0.5,  # Monday
            2: 0.5,  # Tuesday
            3: 0.5,  # Wednesday
            4: 0.6,  # Thursday
            5: 1.0,  # Friday
            6: 1.0   # Saturday
        }
        
        arrival_probs = {}
        for t in range(1, self.params.T + 1):
            base_probs = []
            for arrival, departure in self.booking_classes:
                # Calculate basic factors
                los = departure - arrival + 1
                days_until_arrival = arrival + (self.params.T - t)
                
                # Calculate day-of-week effect
                stay_days = range(arrival, departure + 1)
                stay_dow_factors = [dow_factors[((d-1) % 7)] for d in stay_days]
                avg_dow_factor = np.mean(stay_dow_factors)
                
                # Calculate base probability
                los_factor = np.exp(-0.2 * (los - 1))
                time_factor = np.exp(-0.1 * days_until_arrival)
                base_prob = los_factor * time_factor * avg_dow_factor
                base_probs.append(base_prob)
            
            # Apply scenario adjustments
            base_probs = np.array(base_probs) * scenario['demand_multiplier']
            
            # Apply peak period effects if any
            for peak_t, peak_multiplier in scenario['peak_periods']:
                distance = abs(t - peak_t)
                if distance <= 2:  # Effect spans 5 periods
                    peak_effect = peak_multiplier * (1 - 0.3 * distance)
                    base_probs *= peak_effect
            
            # Add random variation
            raw_probs = base_probs * (1 + self.rng.uniform(
                -scenario['variability'],
                scenario['variability'],
                len(base_probs)
            ))
            
            # Normalize to ensure sum < 1
            total_prob = np.sum(raw_probs)
            if total_prob > 0:
                scale_factor = 0.95 / max(total_prob, 1)  # Ensure sum is at most 0.95
                scaled_probs = raw_probs * scale_factor
            else:
                scaled_probs = raw_probs
            
            arrival_probs[t] = {
                class_: prob for class_, prob in zip(self.booking_classes, scaled_probs)
            }

        # Log scenario statistics
        self._log_arrival_probability_stats(arrival_probs, demand_scenario)
#         daily_arrival_probs = [sum(probs.values()) for probs in arrival_probs.values()]
#         logger.info(f"\nDemand Scenario: {demand_scenario}")
#         logger.info(f"Average daily arrival probability: {np.mean(daily_arrival_probs):.3f}")
#         logger.info(f"Maximum daily arrival probability: {np.max(daily_arrival_probs):.3f}")
#         logger.info(f"Minimum daily arrival probability: {np.min(daily_arrival_probs):.3f}")
        
#         # Log day-of-week statistics
#         dow_probs = {dow: [] for dow in range(7)}
#         for t, probs in arrival_probs.items():
#             for (arrival, departure), prob in probs.items():
#                 dow = (arrival - 1) % 7
#                 dow_probs[dow].append(prob)
        
#         logger.info("\nAverage arrival probabilities by day of week:")
#         for dow in range(7):
#             day_name = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'][dow]
#             avg_prob = np.mean(dow_probs[dow]) if dow_probs[dow] else 0
#             logger.info(f"{day_name}: {avg_prob:.3f}")
        
        return arrival_probs
    
    def _log_arrival_probability_stats(self, arrival_probs: Dict, demand_scenario: str):
        """Log detailed statistics for arrival probabilities."""
        daily_arrival_probs = [sum(probs.values()) for probs in arrival_probs.values()]
        logger.info(f"\nDemand Scenario: {demand_scenario}")
        logger.info(f"Average daily arrival probability: {np.mean(daily_arrival_probs):.3f}")
        logger.info(f"Maximum daily arrival probability: {np.max(daily_arrival_probs):.3f}")
        logger.info(f"Minimum daily arrival probability: {np.min(daily_arrival_probs):.3f}")
        
        # Log day-of-week statistics
        dow_probs = {dow: [] for dow in range(7)}
        for t, probs in arrival_probs.items():
            for (arrival, departure), prob in probs.items():
                dow = (arrival - 1) % 7
                dow_probs[dow].append(prob)
        
        logger.info("\nAverage arrival probabilities by day of week:")
        for dow in range(7):
            day_name = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'][dow]
            avg_prob = np.mean(dow_probs[dow]) if dow_probs[dow] else 0
            logger.info(f"{day_name}: {avg_prob:.3f}")
    
#     def generate_reservation_price_params(self) -> Dict[Tuple[int, int], float]:
#         """
#         Generate epsilon parameters for the linear reservation price survival function.
#         F̄(p) = 1 - ε * p for each booking class.
        
#         Returns:
#             Dictionary mapping booking class to epsilon parameter
#         """
#         # Generate epsilon values ensuring F̄(p_max) >= 0
#         max_epsilon = 1 / self.params.price_max  # Ensures F̄(p_max) >= 0
#         epsilons = {
#             class_: self.rng.uniform(0, max_epsilon)
#             for class_ in self.booking_classes
#         }
        
#         logger.info("Generated reservation price parameters")
#         return epsilons
    def generate_reservation_price_params(self, market_condition: str = 'standard') -> Dict[Tuple[int, int], float]:
        """
        Generate epsilon parameters for the linear reservation price survival function.
        F̄_b(p_t^b) = 1 - ε_b * p_t^b for each booking class.
        """
        if market_condition not in ['standard', 'luxury', 'budget', 'peak_season', 'competitive']:
            raise ValueError(f"Invalid market condition: {market_condition}")

        # Calculate maximum allowable epsilon to ensure F̄_b(p_max) ≥ 0
        max_epsilon = 1 / self.params.price_max

        # Define market-specific parameters
        market_params = {
            'standard': {
                'min_epsilon_factor': 0.5,    # 1/(2*price_max)
                'base_sensitivity': 0.5,      # Start at 50% of range
                'los_factor': 0.10,          # 10% increase per night
                'weekend_premium': 0.20,      # 20% higher for weekend stays
                'random_variation': 0.10      # ±10% random variation
            },
            'luxury': {
                'min_epsilon_factor': 0.25,   # 1/(4*price_max)
                'base_sensitivity': 0.4,      # Start at 40% of range
                'los_factor': 0.05,          # 5% increase per night
                'weekend_premium': 0.10,      # 10% higher for weekend stays
                'random_variation': 0.08      # ±8% random variation
            },
            'budget': {
                'min_epsilon_factor': 0.75,   # 3/(4*price_max)
                'base_sensitivity': 0.6,      # Start at 60% of range
                'los_factor': 0.15,          # 15% increase per night
                'weekend_premium': 0.30,      # 30% higher for weekend stays
                'random_variation': 0.12      # ±12% random variation
            },
            'peak_season': {
                'min_epsilon_factor': 0.4,    # 2/(5*price_max)
                'base_sensitivity': 0.3,      # Start at 30% of range
                'los_factor': 0.08,          # 8% increase per night
                'weekend_premium': 0.10,      # 10% higher for weekend stays
                'random_variation': 0.10      # ±10% random variation
            },
            'competitive': {
                'min_epsilon_factor': 0.6,    # 3/(5*price_max)
                'base_sensitivity': 0.7,      # Start at 70% of range
                'los_factor': 0.12,          # 12% increase per night
                'weekend_premium': 0.25,      # 25% higher for weekend stays
                'random_variation': 0.15      # ±15% random variation
            }
        }

        params = market_params[market_condition]
        min_epsilon = max_epsilon * params['min_epsilon_factor']
        
        epsilons = {}
        for arrival, departure in self.booking_classes:
            # Calculate length of stay
            los = departure - arrival + 1
            
            # Calculate weekend ratio
            stay_days = range(arrival, departure + 1)
            weekend_days = sum(1 for d in stay_days if (d-1) % 7 in [5, 6])
            weekend_ratio = weekend_days / los
            
            # Calculate base epsilon
            base_epsilon = min_epsilon + (max_epsilon - min_epsilon) * (
                params['base_sensitivity'] +
                params['los_factor'] * (los - 1) +
                params['weekend_premium'] * weekend_ratio
            )
            
            # Add random variation
            epsilon = base_epsilon * (1 + self.rng.uniform(
                -params['random_variation'],
                params['random_variation']
            ))
            
            # Ensure epsilon stays within valid bounds
            epsilon = min(max_epsilon, max(min_epsilon, epsilon))
            epsilons[(arrival, departure)] = epsilon
        
        # Log statistics
        self._log_reservation_price_stats(epsilons, market_condition, params)
        return epsilons
    
    def _log_reservation_price_stats(self, epsilons: Dict[Tuple[int, int], float], market_condition: str, market_params: dict):
        """Log detailed statistics for reservation price parameters."""
        logger.info(f"\nReservation Price Parameters for {market_condition.upper()} Market:")
        logger.info(f"Market Characteristics:")
        logger.info(f"- Base Price Sensitivity: {market_params['base_sensitivity']:.2f}")
        logger.info(f"- Length of Stay Factor: {market_params['los_factor']:.3f}")
        logger.info(f"- Weekend Premium: {market_params['weekend_premium']:.2f}")
        logger.info(f"- Random Variation: ±{market_params['random_variation']:.2f}")
        
        logger.info("\nEpsilon Statistics:")
        logger.info(f"Average epsilon: {np.mean(list(epsilons.values())):.6f}")
        logger.info(f"Maximum epsilon: {np.max(list(epsilons.values())):.6f}")
        logger.info(f"Minimum epsilon: {np.min(list(epsilons.values())):.6f}")
        
        # Log average epsilon by length of stay
        los_epsilons = {}
        for (arrival, departure), epsilon in epsilons.items():
            los = departure - arrival + 1
            if los not in los_epsilons:
                los_epsilons[los] = []
            los_epsilons[los].append(epsilon)
        
        logger.info("\nAverage epsilon by length of stay:")
        for los in sorted(los_epsilons.keys()):
            avg_epsilon = np.mean(los_epsilons[los])
            logger.info(f"LOS {los}: {avg_epsilon:.6f}")
            
        # Log average epsilon by day of week
        dow_epsilons = {i: [] for i in range(7)}
        for (arrival, departure), epsilon in epsilons.items():
            stay_days = range(arrival, departure + 1)
            for day in stay_days:
                dow = (day - 1) % 7
                dow_epsilons[dow].append(epsilon)
        
        logger.info("\nAverage epsilon by day of week:")
        days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
        for dow in range(7):
            avg_epsilon = np.mean(dow_epsilons[dow]) if dow_epsilons[dow] else 0
            logger.info(f"{days[dow]}: {avg_epsilon:.6f}")
    
    def generate_initial_prices(self) -> Dict[int, Dict[int, float]]:
        """
        Generate initial prices for each day in the service horizon and each time period.
        
        Returns:
            Dictionary mapping time period to {day: price}
        """
        prices = {}
        for t in range(1, self.params.T + 1):
            prices[t] = {
                day: self.rng.uniform(self.params.price_min, self.params.price_max)
                for day in range(1, self.params.N + 1)
            }
        
        logger.info(f"Generated initial prices for {self.params.T} time periods")
        return prices
    
    def generate_study_instance(self, demand_scenario: str = 'base', market_condition: str = 'standard') -> Dict:
        """
        Generate a complete instance for the hotel dynamic pricing computational study.

        This function creates a comprehensive study instance by combining:
        1. Basic parameters (booking horizon, service horizon, capacity)
        2. Generated booking classes with maximum 7-night stays
        3. Arrival probabilities based on specified demand scenario
        4. Reservation price parameters based on market condition
        5. Initial prices for the booking horizon

        Args:
            demand_scenario: Type of demand pattern to generate. Options include:
                - 'base': Standard demand patterns
                - 'high': Increased arrival probabilities (1.5x base)
                - 'low': Decreased arrival probabilities (0.5x base)
                - 'peak': Time-dependent demand spikes
                - 'fluctuating': Higher demand variability

            market_condition: Type of market condition to model. Options include:
                - 'standard': Balanced price sensitivity
                - 'luxury': Lower price sensitivity
                - 'budget': Higher price sensitivity
                - 'peak_season': Temporarily reduced sensitivity
                - 'competitive': Market-driven high sensitivity

        Returns:
            Dictionary containing:
            - parameters: StudyParameters object with basic configuration
            - booking_classes: List of (arrival, departure) tuples
            - arrival_probabilities: Dict mapping periods to booking class probabilities
            - reservation_price_params: Dict mapping booking classes to epsilon values
            - initial_prices: Dict mapping periods to daily prices
            - scenario_info: Dict containing scenario configuration details
            - generation_timestamp: Creation time of the instance
        """
        instance = {
            'parameters': self.params,
            'booking_classes': self.booking_classes,
            'arrival_probabilities': self.generate_arrival_probabilities(demand_scenario),
            'reservation_price_params': self.generate_reservation_price_params(market_condition),
            'initial_prices': self.generate_initial_prices(),
            'scenario_info': {
                'demand_scenario': demand_scenario,
                'market_condition': market_condition,
                'description': f"{market_condition.capitalize()} market with {demand_scenario} demand"
            },
            'generation_timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        logger.info(f"\nGenerated study instance:")
        logger.info(f"Demand Scenario: {demand_scenario}")
        logger.info(f"Market Condition: {market_condition}")
        logger.info(f"Booking Classes: {len(instance['booking_classes'])}")
        logger.info(f"Time Horizon: T={self.params.T}, N={self.params.N}")

        return instance

In [6]:
def create_test_instance(demand_scenario: str = 'base', 
                     market_condition: str = 'standard', 
                     seed: int = 42) -> Dict:
    """
    Create a test instance for the hotel dynamic pricing computational study.
    
    This function generates a small but realistic test instance that can be used
    for initial validation and testing of the pricing algorithm. It uses a one-week
    service horizon with two weeks of advance booking, maintaining an industry-standard
    capacity ratio of approximately 80%.
    
    Args:
        demand_scenario: Type of demand pattern to test. Default is 'base'.
                       See generate_study_instance() for all options.
        market_condition: Type of market condition to test. Default is 'standard'.
                        See generate_study_instance() for all options.
        seed: Random seed for reproducibility. Default is 42.
    
    Returns:
        Dictionary containing a complete test instance with scenario information
        and generation parameters.
    """
    # Create parameters for a one-week test instance
    params = StudyParameters(
        T=14,      # Two weeks of booking horizon
        N=7,       # One week of service horizon
        C=6,       # 6 rooms (approximately 80% of N)
        price_min=100.0,  # Standard price range for mid-scale hotels
        price_max=500.0,
        alpha=0.1,  # Smoothing parameter for price acceptance
        beta=0.1    # Smoothing parameter for capacity
    )
    
    # Generate test instance with specified scenarios
    generator = DataGenerator(params, seed=seed)
    test_instance = generator.generate_study_instance(
        demand_scenario=demand_scenario,
        market_condition=market_condition
    )
    
    # Log detailed test instance information
    logger.info("\nTest Instance Generation Summary")
    logger.info("================================")
    logger.info("Basic Configuration:")
    logger.info(f"- Booking Horizon (T): {test_instance['parameters'].T} periods")
    logger.info(f"- Service Horizon (N): {test_instance['parameters'].N} days")
    logger.info(f"- Room Capacity (C): {test_instance['parameters'].C} rooms")
    logger.info(f"- Price Range: ${test_instance['parameters'].price_min:.2f} - ${test_instance['parameters'].price_max:.2f}")
    
    logger.info("\nScenario Configuration:")
    logger.info(f"- Demand Scenario: {test_instance['scenario_info']['demand_scenario']}")
    logger.info(f"- Market Condition: {test_instance['scenario_info']['market_condition']}")
    logger.info(f"- Scenario Description: {test_instance['scenario_info']['description']}")
    
    logger.info("\nInstance Statistics:")
    logger.info(f"- Number of Booking Classes: {len(test_instance['booking_classes'])}")
    logger.info(f"- Generation Seed: {seed}")
    logger.info(f"- Generated at: {test_instance['generation_timestamp']}")
    
    # Print summary to console for immediate feedback
    print("\nTest Instance Successfully Generated:")
    print(f"- Scenario: {test_instance['scenario_info']['description']}")
    print(f"- Time Horizon: T={params.T}, N={params.N}")
    print(f"- Booking Classes: {len(test_instance['booking_classes'])}")
    
    return test_instance

In [7]:
if __name__ == "__main__":
    # Generate test instances for different scenarios
    print("\nHotel Dynamic Pricing Study - Test Instance Generation")
    print("=" * 50)
    
    # Generate base scenario test instance
    test_instance = create_test_instance(demand_scenario='base', market_condition='standard')
    
    print("\nInstance Configuration")
    print("-" * 30)
    print(f"Service Horizon (N): {test_instance['parameters'].N} days")
    print(f"Booking Horizon (T): {test_instance['parameters'].T} periods")
    print(f"Room Capacity (C): {test_instance['parameters'].C} rooms")
    print(f"Price Range: ${test_instance['parameters'].price_min:.2f} - ${test_instance['parameters'].price_max:.2f}")
    
    print("\nBooking Classes Analysis")
    print("-" * 30)
    booking_classes = test_instance['booking_classes']
    los_distribution = {}
    for arrival, departure in booking_classes:
        los = departure - arrival + 1
        los_distribution[los] = los_distribution.get(los, 0) + 1
    
    print(f"Total Booking Classes: {len(booking_classes)}")
    print("\nLength of Stay Distribution:")
    for los, count in sorted(los_distribution.items()):
        print(f"  {los} night(s): {count} classes")
    
    print("\nArrival Probability Analysis")
    print("-" * 30)
    arrival_probs = test_instance['arrival_probabilities']
    
    # Analyze first booking period
    probs_t1 = arrival_probs[1]
    total_prob = sum(probs_t1.values())
    print(f"First Period (t=1) Total Arrival Probability: {total_prob:.3f}")
    
    # Calculate average arrival probabilities across all periods
    avg_total_prob = np.mean([sum(arrival_probs[t].values()) for t in range(1, test_instance['parameters'].T + 1)])
    print(f"Average Total Arrival Probability per Period: {avg_total_prob:.3f}")
    
    print("\nReservation Price Analysis")
    print("-" * 30)
    eps_params = test_instance['reservation_price_params']
    epsilon_values = list(eps_params.values())
    
    print(f"Average Epsilon: {np.mean(epsilon_values):.3f}")
    print(f"Min Epsilon: {np.min(epsilon_values):.3f}")
    print(f"Max Epsilon: {np.max(epsilon_values):.3f}")
    
    print("\nInitial Pricing Analysis")
    print("-" * 30)
    initial_prices = test_instance['initial_prices']
    prices_t1 = initial_prices[1]
    price_values = list(prices_t1.values())
    
    print(f"First Period (t=1) Pricing Summary:")
    print(f"  Average Price: ${np.mean(price_values):.2f}")
    print(f"  Minimum Price: ${np.min(price_values):.2f}")
    print(f"  Maximum Price: ${np.max(price_values):.2f}")
    
    print("\nGeneration Information")
    print("-" * 30)
    print(f"Scenario: {test_instance['scenario_info']['description']}")
    print(f"Generated at: {test_instance['generation_timestamp']}")

INFO:__main__:Generated 28 booking classes with max LOS = 7
INFO:__main__:Initialized DataGenerator with 28 booking classes
INFO:__main__:
Demand Scenario: base
INFO:__main__:Average daily arrival probability: 0.950
INFO:__main__:Maximum daily arrival probability: 0.950
INFO:__main__:Minimum daily arrival probability: 0.950
INFO:__main__:
Average arrival probabilities by day of week:
INFO:__main__:Sunday: 0.035
INFO:__main__:Monday: 0.028
INFO:__main__:Tuesday: 0.029
INFO:__main__:Wednesday: 0.031
INFO:__main__:Thursday: 0.038
INFO:__main__:Friday: 0.051
INFO:__main__:Saturday: 0.049
INFO:__main__:
Reservation Price Parameters for STANDARD Market:
INFO:__main__:Market Characteristics:
INFO:__main__:- Base Price Sensitivity: 0.50
INFO:__main__:- Length of Stay Factor: 0.100
INFO:__main__:- Weekend Premium: 0.20
INFO:__main__:- Random Variation: ±0.10
INFO:__main__:
Epsilon Statistics:
INFO:__main__:Average epsilon: 0.001732
INFO:__main__:Maximum epsilon: 0.002000
INFO:__main__:Minimum e


Hotel Dynamic Pricing Study - Test Instance Generation

Test Instance Successfully Generated:
- Scenario: Standard market with base demand
- Time Horizon: T=14, N=7
- Booking Classes: 28

Instance Configuration
------------------------------
Service Horizon (N): 7 days
Booking Horizon (T): 14 periods
Room Capacity (C): 6 rooms
Price Range: $100.00 - $500.00

Booking Classes Analysis
------------------------------
Total Booking Classes: 28

Length of Stay Distribution:
  1 night(s): 7 classes
  2 night(s): 6 classes
  3 night(s): 5 classes
  4 night(s): 4 classes
  5 night(s): 3 classes
  6 night(s): 2 classes
  7 night(s): 1 classes

Arrival Probability Analysis
------------------------------
First Period (t=1) Total Arrival Probability: 0.950
Average Total Arrival Probability per Period: 0.950

Reservation Price Analysis
------------------------------
Average Epsilon: 0.002
Min Epsilon: 0.001
Max Epsilon: 0.002

Initial Pricing Analysis
------------------------------
First Period (t=